In [1]:
import torch, torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=21).to(device)
model.train();

In [2]:
from src.voc_utils import VOCDatasetFORREAL, collate_fn

train_set = VOCDatasetFORREAL(root=f'data/torchvision/pascal_2007_trainval', image_set='trainval')
test_set  = VOCDatasetFORREAL(root=f'data/torchvision/pascal_2007_test', image_set='test')
# train_set = VOCDetection(root=f'data/torchvision/pascal_2007_trainval', year='2007', image_set='trainval', download=True, transforms=transforms)
# test_set = VOCDetection(root=f'data/torchvision/pascal_2007_test', year='2007', image_set='test', download=True, transform=ToTensor())

train_loader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_loader  = torch.utils.data.DataLoader(test_set, batch_size=2, shuffle=True, num_workers=0, collate_fn=collate_fn)

Using downloaded and verified file: data/torchvision/pascal_2007_trainval\VOCtrainval_06-Nov-2007.tar
Extracting data/torchvision/pascal_2007_trainval\VOCtrainval_06-Nov-2007.tar to data/torchvision/pascal_2007_trainval
Using downloaded and verified file: data/torchvision/pascal_2007_test\VOCtest_06-Nov-2007.tar
Extracting data/torchvision/pascal_2007_test\VOCtest_06-Nov-2007.tar to data/torchvision/pascal_2007_test


In [3]:
from src.torchvision.references.detection.engine import train_one_epoch, evaluate


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0001, momentum=0.9, weight_decay=0.005)

for epoch in range(5):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)
    evaluate(model, test_loader, device=device)

Epoch: [0]  [   0/2506]  eta: 4:15:21  lr: 0.000000  loss: 3.4918 (3.4918)  loss_classifier: 3.3448 (3.3448)  loss_box_reg: 0.0998 (0.0998)  loss_objectness: 0.0146 (0.0146)  loss_rpn_box_reg: 0.0326 (0.0326)  time: 6.1140  data: 0.0275  max mem: 2113
Epoch: [0]  [1000/2506]  eta: 0:08:03  lr: 0.000100  loss: 0.5676 (1.1476)  loss_classifier: 0.2525 (0.7356)  loss_box_reg: 0.2743 (0.3788)  loss_objectness: 0.0060 (0.0158)  loss_rpn_box_reg: 0.0093 (0.0173)  time: 0.3077  data: 0.0132  max mem: 4022
Epoch: [0]  [2000/2506]  eta: 0:02:39  lr: 0.000100  loss: 0.4622 (0.8779)  loss_classifier: 0.1889 (0.4963)  loss_box_reg: 0.2686 (0.3501)  loss_objectness: 0.0021 (0.0148)  loss_rpn_box_reg: 0.0159 (0.0168)  time: 0.3172  data: 0.0132  max mem: 4022
Epoch: [0]  [2505/2506]  eta: 0:00:00  lr: 0.000100  loss: 0.4749 (0.8082)  loss_classifier: 0.1538 (0.4370)  loss_box_reg: 0.2399 (0.3394)  loss_objectness: 0.0031 (0.0147)  loss_rpn_box_reg: 0.0122 (0.0171)  time: 0.3044  data: 0.0126  max me

In [4]:
{
    'annotation': {
        'folder': 'VOC2007', 
        'filename': '000005.jpg', 
        'source': {
            'database': 'The VOC2007 Database', 
            'annotation': 'PASCAL VOC2007', 
            'image': 'flickr', 
            'flickrid': '325991873'
        }, 
        'owner': {
            'flickrid': 'archintent louisville', 
            'name': '?'
        }, 
        'size': {
            'width': '500', 
            'height': '375', 
            'depth': '3'
        }, 
        'segmented': '0', 
        'object': [
            {
                'name': 'chair', 
                'pose': 'Rear', 
                'truncated': '0', 
                'difficult': '0', 'bndbox': {'xmin': '263', 'ymin': '211', 'xmax': '324', 'ymax': '339'}}, {'name': 'chair', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '165', 'ymin': '264', 'xmax': '253', 'ymax': '372'}}, {'name': 'chair', 'pose': 'Unspecified', 'truncated': '1', 'difficult': '1', 'bndbox': {'xmin': '5', 'ymin': '244', 'xmax': '67', 'ymax': '374'}}, {'name': 'chair', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0', 'bndbox': {'xmin': '241', 'ymin': '194', 'xmax': '295', 'ymax': '299'}}, {'name': 'chair', 'pose': 'Unspecified', 'truncated': '1', 'difficult': '1', 'bndbox': {'xmin': '277', 'ymin': '186', 'xmax': '312', 'ymax': '220'}}]}}

{'annotation': {'folder': 'VOC2007',
  'filename': '000005.jpg',
  'source': {'database': 'The VOC2007 Database',
   'annotation': 'PASCAL VOC2007',
   'image': 'flickr',
   'flickrid': '325991873'},
  'owner': {'flickrid': 'archintent louisville', 'name': '?'},
  'size': {'width': '500', 'height': '375', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'chair',
    'pose': 'Rear',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '263', 'ymin': '211', 'xmax': '324', 'ymax': '339'}},
   {'name': 'chair',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '165', 'ymin': '264', 'xmax': '253', 'ymax': '372'}},
   {'name': 'chair',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '1',
    'bndbox': {'xmin': '5', 'ymin': '244', 'xmax': '67', 'ymax': '374'}},
   {'name': 'chair',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '241', 'ymin': '194', 'xmax': '295',

In [5]:
import torch
from src.torchvision.references.detection.engine import train_one_epoch, evaluate
import src.torchvision.references.detection.utils as utils


# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

NameError: name 'PennFudanDataset' is not defined